In [1]:
import numpy as np
import pandas as pd
import torch

from pathlib import Path
import sys
sys.path.insert(0, str(Path.cwd().resolve().parents[0] / '2_Propensities'))

import MF_class as MF

np.random.seed(42)
if np.random.choice(np.arange(1000)) != 102:
    raise ValueError("Random seed is not set correctly.")

```
                                USERS                             
         ┌───────────────────────────────────────────────────────┐
         │                          │                            │
         │                          │                            │
         │                          │                            │
         │                          │                            │
ITEMS    │                          │                            │
         │                          │                            │
         │                          │                            │
         ├──────────────────────────┼────────────────────────────┤
         │                          │████████████████████████████│
         │                          │████████████████████████████│
         └───────────────────────────────────────────────────────┘
```

# 1. Load Data

In [2]:
base_artifacts = Path.cwd().resolve().parents[1] / 'CausalI2I_artifacts'

train = pd.read_csv(base_artifacts / 'Datasets' / 'Simulation' / 'train.csv')
test = pd.read_csv(base_artifacts / 'Datasets' / 'Simulation' / 'test.csv')

n_users = train['user_id'].nunique()
n_items = train['item_id'].nunique()
print(f'Number of users: {n_users}, Number of items: {n_items}')

Number of users: 6040, Number of items: 3952


# 2. Train Model

In [10]:
model = MF.MatrixFactorizationTorch(n_users, n_items, n_factors=20)
model.fit(
    train_data=train.values,
    val_data=test.values,
    lr=2e-3, 
    wd=1e-7,
    pos_weight=1,
    batch_size=2**15,
    n_epochs=50,
    device=torch.device('cuda:0'), 
    use_amp=True)

Epoch  ||- - - - - - - - Train - - - - - - - -||- - - - - - Validation - - - - - - - || Epoch's | COS θ | Time     
Number || BCE    | BCE-POS | BCE-NEG | MPR    || BCE    | BCE-POS | BCE-NEG | MPR    || Change  |       | Elapsed  
=======||========|=========|=========|========||========|=========|=========|========||=========|=======|==========
   1   || 0.2339 |  1.8770 |  0.0787 | 0.7968 || 0.2420 |  1.8427 |  0.0843 | 0.7932 || 156.27  | None  | 00:03.68
   2   || 0.2259 |  1.7715 |  0.0798 | 0.8062 || 0.2332 |  1.7555 |  0.0832 | 0.8025 ||  41.87  | 0.250 | 00:07.40
   3   || 0.2221 |  1.7389 |  0.0788 | 0.8119 || 0.2304 |  1.7315 |  0.0825 | 0.8063 ||  28.09  | 0.440 | 00:11.12
   4   || 0.2183 |  1.7051 |  0.0778 | 0.8177 || 0.2273 |  1.7073 |  0.0815 | 0.8106 ||  27.27  | 0.805 | 00:14.81
   5   || 0.2155 |  1.6818 |  0.0769 | 0.8218 || 0.2252 |  1.6921 |  0.0807 | 0.8135 ||  22.76  | 0.753 | 00:18.53
   6   || 0.2130 |  1.6587 |  0.0764 | 0.8253 || 0.2235 |  1.6719 |  0.0808 |

### Save Model

In [ ]:
model.save(path=base_artifacts / 'Propensity_Models' / 'MF20_simulation.pt', note=None)

### Load Model

In [3]:
loaded_model = MF.MatrixFactorizationTorch(n_users, n_items, n_factors=20)
loaded_model.load(path=base_artifacts / 'Propensity_Models' / 'MF20_simulation.pt')

Loaded model summary:
Model:                      MatrixFactorizationTorch
Number of users:            6040
Number of items:            3952
Number of factors:          20
Learning rate:              0.002
Weight decay:               1e-07
Positive weight:            1
Batch size:                 32768
Number of epochs:           50
Device:                     cuda:0
Use AMP:                    True
Timestamp:                  2026-01-28 18:28:10
